In [ ]:
import warnings, os
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

In [ ]:
import sys
sys.path.append("/home/renatomz/Documents/Projects/Dal-io")

## Import Base Packages

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as plt

from scipy.stats import skew, kurtosis

## Import Dal-IO Pieces

In [ ]:
from dalio.base.memory import LazyRunner, LocalMemory

from dalio.external import (
    PandasInFile,
    YahooDR, 
    PyPlotGraph, 
    PyPfOptGraph,
    PySubplotGraph
)

from dalio.translator import (
    YahooStockTranslator,
    StockStreamFileTranslator,
)
    
from dalio.pipe import (
    Pipe,
    PipeLine,
    DateSelect,
    ColSelect, 
    Index,
    Change,
    StockReturns, 
    Period, 
    Custom, 
    Rolling,
    CovShrink, 
    ExpectedReturns,
    PandasLinearModel,
    OptimumWeights,
)

from dalio.model import (
    MakeCriticalLine, 
    MakeEfficientFrontier, 
    OptimumPortfolio,
    XYLinearModel,
)

from dalio.application import (
    Grapher,
    PandasTSGrapher,
    PandasMultiGrapher,
    LMGrapher,
)

from dalio.ops import risk_metrics

## Base Setup

In [ ]:
yahoo_in = YahooStockTranslator()(YahooDR())
yahoo = ColSelect(columns="close")(yahoo_in)

In [ ]:
stocks = StockStreamFileTranslator()(PandasInFile("sample_stocks.xlsx"))

In [ ]:
raw_in = Pipe()(stocks)

In [ ]:
time = DateSelect()
time.set_start("1999-12-31")
time.set_end("2019-12-31")

mem = LazyRunner(LocalMemory, buff=4, update=True)(raw_in) # 11.09 sec
# mem = raw_in # 28.14 sec
price = PipeLine(time)(mem)

In [ ]:
simple_rets = price + Change(strategy="pct_change")

# ticker = ["WMT", "MSFT", "AMZN", "HD"]
# ticker = ["AMZN", "HD", "NVDA", "RL"]
ticker = ["NVDA", "RL", "GPS", "WMT"]
# ticker = ["GPS", "MSFT", "AMZN", "NVDA"]

In [ ]:
price.run(ticker=ticker)

## Part A

### Setup

In [ ]:
daily_var = simple_rets + Custom(risk_metrics, 0.94)
daily_vol = daily_var + Custom(lambda x: np.sqrt(x))

In [ ]:
daily_vol.run(ticker=ticker)

In [ ]:
ret_avg = simple_rets + Custom(np.mean)
ret_sd = simple_rets + Custom(np.std)
ret_skew = simple_rets + Custom(skew)
ret_kurt = simple_rets + Custom(kurtosis)

In [ ]:
ret_avg.run(ticker=ticker)

In [ ]:
summary_grapher = PandasMultiGrapher(2, 2)\
    .set_input((0, 0), price)\
        .set_piece((0, 0), "line", x_index=True, y="price", lw=0.5)\
    .set_input((0, 1), simple_rets)\
        .set_piece((0, 1), "line", x_index=True, y="price", lw=0.3)\
    .set_input((1, 0), daily_vol)\
        .set_piece((1, 0), "line", x_index=True, y="price", lw=0.5)\
    .set_input((1, 1), simple_rets)\
        .set_piece((1, 1), "histogram", x="price", bins=30)\
    .set_output("data_out", PySubplotGraph(2, 2, figsize=(16, 8)))

fig = summary_grapher.run(ticker="WMT")

## Part B

### Setup

In [ ]:
p = price.run(ticker=ticker)

In [ ]:
p.resample("Y").apply(lambda x: x[-1])

In [ ]:
annual_rets = price + Period("Y", agg_func=lambda x: (x[-1] - x[0])/x[0])

In [ ]:
price

In [ ]:
annual_rets = price + \
    Period("Y", agg_func=lambda x: x[-1]) + \
    Change(strategy="pct_change", rm_first=True)

In [ ]:
annual_rets.run(ticker=ticker)

In [ ]:
avg_rets = simple_rets + Period("Y", agg_func=np.mean)
var_rets = simple_rets + Period("Y", agg_func=np.var)
std_rets = simple_rets + Period("Y", agg_func=np.std)

In [ ]:
ann_agg = annual_rets + Custom(np.mean)

In [ ]:
ann_agg.run(ticker=ticker)

In [ ]:
cov_rets = annual_rets + Custom(lambda x: x.cov(), strategy="pipe")
corr_rets = annual_rets + Custom(lambda x: x.corr(), strategy="pipe")

In [ ]:
cov_rets.run(ticker=ticker)

In [ ]:
S = CovShrink(frequency=252)(price)\
    .set_piece("shrinkage", "ledoit_wolf")

mu = ExpectedReturns()(price)\
    .set_piece("return_model", "james_stein_shrinkage")

In [ ]:
cla_lo = MakeCriticalLine(weight_bounds=(0.05, 1))\
    .set_input("sample_covariance", cov_rets)\
    .set_input("expected_returns", ann_agg)

In [ ]:
res = cla_lo.run(ticker=ticker)

In [ ]:
res.efficient_frontier(points=100)

In [ ]:
cla_graph = Grapher()\
    .set_input("data_in", cla_lo)\
    .set_output("data_out", PyPfOptGraph(figsize=(12, 8)))

In [ ]:
cla_graph.run(ticker=ticker)

## Part C

### Setup Efficient Frontier

In [ ]:
ef_lo = MakeEfficientFrontier(weight_bounds=(0.05, 1))\
    .set_input("sample_covariance", cov_rets)\
    .set_input("expected_returns", ann_agg)

ef_ls = MakeEfficientFrontier(weight_bounds=(-1, 1))\
    .set_input("sample_covariance", cov_rets)\
    .set_input("expected_returns", ann_agg)

#### Get long-only efficient frontier

In [ ]:
lo_res = ef_lo.run(ticker=ticker)

In [ ]:
lo_res.max_sharpe(risk_free_rate=0.0)

In [ ]:
lo_res.portfolio_performance()

#### Get long-short efficient frontier

In [ ]:
ls_res = ef_ls.run(ticker=ticker)

In [ ]:
ls_res.max_sharpe(risk_free_rate=0.0)

In [ ]:
ls_res.portfolio_performance()

#### Get optimum weights

In [ ]:
max_shp_weights_lo = OptimumWeights()(ef_lo)\
    .set_piece("strategy", "max_sharpe", risk_free_rate=0.0)

max_shp_weights_ls = OptimumWeights()(ef_ls)\
    .set_piece("strategy", "max_sharpe", risk_free_rate=0.0)

min_vol_weights_lo = max_shp_weights_lo.with_piece("strategy", "min_volatility")

min_vol_weights_ls = max_shp_weights_ls.with_piece("strategy", "min_volatility")

In [ ]:
max_shp_weights_lo.run(ticker=ticker)

In [ ]:
max_shp_weights_ls.run(ticker=ticker)

In [ ]:
min_vol_weights_lo.run(ticker=ticker)

In [ ]:
min_vol_weights_ls.run(ticker=ticker)

## Part D

### Setup

In [ ]:
two_years = DateSelect()\
    .set_start("2018-01-01")\
    .set_end("2019-12-31")

price_2y = PipeLine(two_years)(mem)
rets_2y = simple_rets.with_input(price_2y)

In [ ]:
lm = PandasLinearModel()\
    .set_input(Index(100)(price_2y))\
    .set_piece("strategy", "LinearRegression")

In [ ]:
res = lm.run(ticker=ticker)

In [ ]:
res.intercept_

In [ ]:
res.coef_

In [ ]:
all_lm_graph = LMGrapher(legend="upper left")\
    .set_input("data_in", Index(100)(price_2y))\
    .set_input("linear_model", lm)\
    .set_output("data_out", PyPlotGraph(figsize=(12, 8)))

In [ ]:
all_lm_graph.run(ticker=ticker)

In [ ]:
comp_tick = "WMT"

spy = ColSelect(("price", "SPY"))(rets_2y)
comp = ColSelect(("price", comp_tick))(rets_2y)

 # Using simple_rets for full data optimization
opt_port = OptimumPortfolio()\
    .set_input("data_in", simple_rets)\
    .set_input("weights_in", max_shp_weights_lo)

sp500_lm = XYLinearModel()\
    .set_input("x", spy)\
    .set_input("y", two_years(opt_port))\
    .set_piece("strategy", "LinearRegression")

lm_res = sp500_lm.run(ticker=[comp_tick, "SPY"])

In [ ]:
opt_port.run(ticker=ticker)

In [ ]:
lm_res.coef_

In [ ]:
lm_res.score(
    spy.run(), 
    comp.run()
)

In [ ]:
sp500_lm_graph = LMGrapher(x=("price", "SPY"), y=("price", "WMT"))\
    .set_input("data_in", rets_2y)\
    .set_input("linear_model", sp500_lm)\
    .set_output("data_out", PyPlotGraph(figsize=(12, 8)))

In [ ]:
sp500_lm_graph.run(ticker=["WMT", "SPY"])

## Part E

### Setup

In [ ]:
sp500_const = MakeEfficientFrontier(weight_bounds=(0.05, 1))\
    .set_input("sample_covariance", cov_rets)\
    .set_input("expected_returns", ann_agg)\
    .add_stock_weight_constraint(ticker="SPY", comparisson="==", weight=0.6)

In [ ]:
res = sp500_const.run(ticker=ticker+["SPY"])

In [ ]:
res.max_sharpe(risk_free_rate=0.0)

In [ ]:
res.portfolio_performance()

In [ ]:
sp_opt_weights = max_shp_weights_lo.with_input(sp500_const)

In [ ]:
sp_opt_port = opt_port\
    .with_input("weights_in", sp_opt_weights)\
    .set_input("data_in", annual_rets)

In [ ]:
sp_opt_port.run(ticker=ticker+["SPY"])

In [ ]:
simple_rets.run(ticker="SPY")